# Example notebook for WCS bug ans mask plotting bug

In [17]:
from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay
from IPython.display import IFrame
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage
import gc

In [20]:
def mask_and_var_plots (datasetRefs, matplot=None, display_num=None):
    """
    Create an array of plots with their mask and variance

    Parameters
    ----------
    datasetRefs: a set of DataSetRefs
    returned by the butler
    matplot: Can be set to True
    display_num: Integer setting number of images

    Returns
    -------
    None
    """
    
    if not display_num:
        display_num=1
    else:
        display_num=display_num
    
    if matplot != None:
        afwDisplay.setDefaultBackend('matplotlib') 
        
        for i, ref in enumerate(datasetRefs):

            calexp = butler.getDirect(ref)
            print('Visit: ', calexp.visitInfo.getId(), ', Detector: ',calexp.detector.getId())
            fig, ax = plt.subplots(1, 3, figsize=(14, 7))
            plt.sca(ax[0])  # set the first axis as current
            plt.xlabel('Image with Mask Overlay')
            display1 = afwDisplay.Display(frame=fig)
            display1.scale('asinh', -1, 30)
            display1.mtv(calexp)
            plt.sca(ax[1])  # set the second axis as current
            display2 = afwDisplay.Display(frame=fig)
            display2.mtv(calexp.mask)
            plt.tight_layout()
            plt.sca(ax[2]) 
            display3 = afwDisplay.Display(frame=fig)
            display3.scale('asinh', 'zscale')
            display3.mtv(calexp.variance)
            plt.tight_layout()
            plt.show()
            remove_figure(fig)
            
            if i == display_num:
                print('...')
                break

    else:
        afwDisplay.setDefaultBackend('firefly')
        for i, ref in enumerate(datasetRefs):

            calexp = butler.getDirect(ref)
            print('Visit: ', calexp.visitInfo.getId(), ', Detector: ',calexp.detector.getId())
            display = afwDisplay.Display(frame=3*i)
            display.scale('asinh', -1, 30)
            display.mtv(calexp)
            display = afwDisplay.Display(frame=3*i+1)
            display.setMaskTransparency(90)
            display.mtv(calexp.mask)
            display = afwDisplay.Display(frame=3*i+2)
            display.scale('asinh', 'zscale')
            display.mtv(calexp.variance)

            if i == display_num:
                print('...')
                break

In [3]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [22]:
collection="u/elhoward/DM-35285/hsc-output-collection"
butler = lsst.daf.butler.Butler("/repo/main", collections=collection)

registry = butler.registry
dt = registry.getDatasetType('calexp')

datasetType = 'calexp'
datasetRefs = set(registry.queryDatasets(datasetType, dataId=None))

In [23]:
afwDisplay.setDefaultBackend('firefly')
display = afwDisplay.Display(frame=1)
IFrame(display.getClient().get_firefly_url(), 1100, 700)

In [21]:
mask_and_var_plots(datasetRefs)

Visit:  22608 , Detector:  33
Visit:  22658 , Detector:  101
...
